Packages

In [4]:
import os
import numpy
import pandas as pd
import tqdm
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/28 22:41:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/28 22:41:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/28 22:41:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
file_name_list = sorted(os.listdir('../data/curated/data'))

In [4]:
sdf = read_file(spark, 'data', '../data/curated/')

|> Loading File...


|> Loading Finished!
-RECORD 0-----------------------------------------------
 user_id         | 1                                    
 merchant_abn    | 46451548968                          
 dollar_value    | 72.61581642788431                    
 order_id        | 76bab304-fa2d-4004-8179-8638b56a873e 
 merchant_name   | Tempus Eu Ligula Limited             
 tags            | health and beauty spas               
 take_rate       | 6.04                                 
 type            | a                                    
 consumer_id     | 1195503                              
 name            | Yolanda Williams                     
 address         | 413 Haney Gardens Apt. 742           
 state           | WA                                   
 postcode        | 6935                                 
 gender          | Female                               
 New cases / day | 1,621                                
 order_datetime  | 2021-11-26                           
only showi

In [5]:
# type
type_indexer = StringIndexer(inputCol = "type", outputCol = "typeIndex", handleInvalid="skip")

# gender
gender_indexer = StringIndexer(inputCol = "gender", outputCol = "genderIndex", handleInvalid="skip")

# tag
tag_indexer = StringIndexer(inputCol = "tags",outputCol = "tagIndex", handleInvalid="skip")
onehotencoder_tag_vector = OneHotEncoder(inputCol = "tagIndex", outputCol = "tag_vec")

#Create pipeline and pass all stages
pipeline = Pipeline(stages=[gender_indexer, 
                            type_indexer,
                            tag_indexer,
                            onehotencoder_tag_vector])

In [6]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------------+--------------------+-----+--------+------+---------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|consumer_id|             name|             address|state|postcode|gender|New cases / day|order_datetime|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------------+--------------------+-----+--------+------+---------------+--------------+
|      1| 46451548968| 72.61581642788431|76bab304-fa2d-400...|Tempus Eu Ligula ...|health and beauty...|     6.04|   a|    1195503| Yolanda Williams|413 Haney Gardens...|   WA|    6935|Female|          1,621|    2021-11-26|
|      1| 49167531725| 51.58228625503599|7080c274-17f7-4cc...|     Felis Institute|digital goods: bo...|

In [7]:
sdf_transformed = pipeline.fit(sdf).transform(sdf)
sdf_transformed.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------------+--------------------+-----+--------+------+---------------+--------------+-----------+---------+--------+---------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|consumer_id|             name|             address|state|postcode|gender|New cases / day|order_datetime|genderIndex|typeIndex|tagIndex|        tag_vec|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------------+--------------------+-----+--------+------+---------------+--------------+-----------+---------+--------+---------------+
|      1| 46451548968| 72.61581642788431|76bab304-fa2d-400...|Tempus Eu Ligula ...|health and beauty...|     6.04|   a|    1195503| Yolanda Williams|413 Haney Gardens...|   WA|    6935|Fe

- figure out that tag_vec

In [8]:
SAMPLE_SIZE = 0.05
sdf_transformed = sdf_transformed.sample(SAMPLE_SIZE, seed=0)

In [9]:
df_transformed = sdf_transformed.toPandas()
df_transformed

,user_id,merchant_abn,dollar_value,order_id,merchant_name,tags,take_rate,type,consumer_id,name,address,state,postcode,gender,New cases / day,order_datetime,genderIndex,typeIndex,tagIndex,tag_vec
0,17,18709053174,22.392280,901d96f0-5400-47b8-9b84-f4e662aa4ce8,Massa Rutrum Industries,tent and awning shops,3.39,b,1331093,Alyssa Wilson,44353 Nathan Ridge,VIC,3719,Female,"1,621",2021-11-26,1.0,1.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,52,73499119023,16.299283,5f4af1eb-e9aa-437e-850d-e50694a1eff3,Nisi Dictum Company,"watch, clock, and jewelry repair shops",5.56,a,95154,Wendy Brown,27343 Rodriguez Ports,WA,6963,Female,"1,621",2021-11-26,1.0,0.0,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,170,78982081666,106.189382,056852ba-612a-4dc3-a9f3-53a02ff2d888,Lectus Corp.,"books, periodicals, and newspapers",4.96,b,1430320,Nicole Russo,661 Moore Island,WA,6313,Female,"1,621",2021-11-26,1.0,1.0,12.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,226,94547309480,142.604001,6df86507-6255-4610-a6aa-88474c6c9d93,Lorem Ac Institute,bicycle shops - sales and service,2.66,c,710483,Emily Singh,0107 Flores Street Suite 423,SA,5642,Female,"1,621",2021-11-26,1.0,2.0,18.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,250,78327477486,517.366887,45feae7d-f31f-41f3-8b97-73c24ce779c8,Auctor Inc.,"lawn and garden supply outlets, including nurs...",1.47,c,470403,Robert Ochoa,326 Hodges Square,VIC,3447,Male,"1,621",2021-11-26,0.0,2.0,13.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216722,17961,76767266140,342.948509,7bf22352-e342-4e01-83a0-81ce45219027,Phasellus At Limited,"furniture, home furnishings and equipment shop...",4.65,b,1008461,Rebecca Morse,2629 Debra Parks,SA,5276,Female,"33,079",2022-01-26,1.0,1.0,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
216723,18639,17488304283,76.787254,397cb2b2-c1f5-4a9f-b22d-3a2d31091bda,Posuere Cubilia Curae Corporation,"cable, satellite, and other pay television and...",6.18,a,994231,Mallory Allen,374 Thomas Vista,NSW,4375,Female,"33,079",2022-01-26,1.0,0.0,4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
216724,19189,80324045558,74.506476,a4af9ede-695a-4479-ad98-bc23979eb236,Ipsum Dolor Sit Corporation,"gift, card, novelty, and souvenir shops",1.47,c,1102313,Brandy Robinson,3149 Mary Junction Apt. 899,WA,6923,Female,"33,079",2022-01-26,1.0,2.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
216725,21171,89726005175,45.786481,ec7d731c-c3d9-47f5-8313-fab3d392840a,Est Nunc Consulting,tent and awning shops,6.01,a,562461,Mark Hughes,28321 Davis Groves Apt. 024,WA,6517,Male,"33,079",2022-01-26,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


- postcode latitude and longitude change

According to the size of csv file, choose pandas to do the dataframe slice